# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [26]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('\\Users\\Administrator\\production\\05_src\\data\\adult\\adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
adult_dt.head()


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [27]:
#  Create a dataframe `X` that holds the features (all columns that are not `income`).
X = adult_dt
X= X.drop(columns=['income'])
X.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


In [28]:
# Create a dataframe `Y` that holds the target data (`income`).
Y=adult_dt['income']
Y.head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    1
8    1
9    1
Name: income, dtype: int64

In [29]:
# From `X` and `Y`, obtain the training and testing data sets:
#    - Use a train-test split of 70-30%. 
#    - Set the random state of the splitting function to 42.

#importing train_test_split from sklearn
from sklearn.model_selection import train_test_split

# Use a train-test split of 70% for Training and 30% for Testing        And     Set the random state of the splitting function to 42
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Display the shape of the datasets to verify the split
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)

X_train shape: (22792, 14)
X_test shape: (9769, 14)
Y_train shape: (22792,)
Y_test shape: (9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

**random state** controls how the data is split, so if you define a random state, every time you run the code the data will be splitted the same as first split, 
This helps in comparing results and debugging 
Also helps in sharing the work
Also gurentee your work is re-producable.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [41]:
#Importing needed Libraries
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

# Specify columns that are numerical 
numerical_cols = [
    'age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week'
]

#--------------------------------------------------------------------------------------
# Specify columns that are categorical
categorical_cols = [
    'workclass', 'education', 'marital-status', 'occupation', 'relationship', 
    'race', 'sex', 'native-country'
]


#--------------------------------------------------------------------------------------
# the ColumnTransformer creation 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 
         Pipeline(steps=[
             #As requested in the question KNN with 7 neigbhours and criteria for weighting is Distance
             ('knn_imputer', KNNImputer(n_neighbors=7, weights='distance')), 
             ('scaler', StandardScaler())
         ]), 
         numerical_cols),
        
        ('cat', 
         Pipeline(steps=[
             #As requested in the question Applying Simple Imputer and using onehot encoding
             ('simple_imputer', SimpleImputer(strategy='most_frequent')),
             ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))
         ]), 
         categorical_cols)
    ]
)

# Printting the coulmn transformer
preprocessor

#--------------------------------------------------------------------------------------
# Fit and transform the training data
#X_train_transformed = preprocessor.fit_transform(X_train)

# Transform the test data
#X_test_transformed = preprocessor.transform(X_test)

# Display the shape of the transformed data
#print("Transformed X_train shape:", X_train_transformed.shape)
#print("Transformed X_test shape:", X_test_transformed.shape)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('knn_imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('scaler', StandardScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [42]:
# import necessary Libraries
from sklearn.ensemble import RandomForestClassifier

# Create the full pipeline with preprocessing and classifier
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),  # Preprocessing step
    ('classifier', RandomForestClassifier(random_state=42))  # Classifier step
])

# Printing the pipline
pipeline

# Fit the pipeline on the training data
#pipeline.fit(X_train, Y_train)

# To make predictions on the test data
#Y_pred = pipeline.predict(X_test)

# Display the predictions
#print("Predictions:", Y_pred)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('knn_imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat',
                                                  Pipeline(steps=[('simple_imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [43]:
# importing necessary Libraries
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Perform cross-validation
scoring = {
    'neg_log_loss': make_scorer(log_loss, needs_proba=True),
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}

# Evaluate the model using cross_validatation While no of folds used = 5
cv_results = cross_validate(pipeline, X, Y, scoring=scoring, cv=5, return_train_score=True)

# Report training and validation results
print("Training Results:")
print("Mean Negative Log Loss:", cv_results['train_neg_log_loss'].mean())
print("Mean ROC AUC:", cv_results['train_roc_auc'].mean())
print("Mean Accuracy:", cv_results['train_accuracy'].mean())
print("Mean Balanced Accuracy:", cv_results['train_balanced_accuracy'].mean())

print("\nValidation Results:")
print("Mean Negative Log Loss:", cv_results['test_neg_log_loss'].mean())
print("Mean ROC AUC:", cv_results['test_roc_auc'].mean())
print("Mean Accuracy:", cv_results['test_accuracy'].mean())
print("Mean Balanced Accuracy:", cv_results['test_balanced_accuracy'].mean())



c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(
c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Training Results:
Mean Negative Log Loss: 0.08033909682706751
Mean ROC AUC: 0.9999999895207642
Mean Accuracy: 0.9999769664862376
Mean Balanced Accuracy: 0.9999848300970873

Validation Results:
Mean Negative Log Loss: 0.36390351025150125
Mean ROC AUC: 0.903119389078505
Mean Accuracy: 0.8542121059336629
Mean Balanced Accuracy: 0.7747126844342593


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [45]:
# Create a DataFrame for fold-level results
fold_results = pd.DataFrame({
    'Fold': range(1, 6),
    'Negative Log Loss': cv_results['test_neg_log_loss'],
    'ROC AUC': cv_results['test_roc_auc'],
    'Accuracy': cv_results['test_accuracy'],
    'Balanced Accuracy': cv_results['test_balanced_accuracy']
})

# Sort the results by Negative Log Loss
sorted_results = fold_results.sort_values(by='Negative Log Loss')

# Display the sorted DataFrame
print(sorted_results)

   Fold  Negative Log Loss   ROC AUC  Accuracy  Balanced Accuracy
2     3           0.348820  0.906090  0.852119           0.778059
4     5           0.350301  0.907609  0.857494           0.779421
3     4           0.358108  0.901963  0.857801           0.777664
0     1           0.363171  0.902707  0.853831           0.770151
1     2           0.399118  0.897228  0.849816           0.768268


Calculate the mean of each metric. 

In [46]:
# Calculate and display the mean of each metric
mean_results = sorted_results.mean().to_frame(name='Mean').reset_index()
mean_results.columns = ['Metric', 'Mean']
print(mean_results)

              Metric      Mean
0               Fold  3.000000
1  Negative Log Loss  0.363904
2            ROC AUC  0.903119
3           Accuracy  0.854212
4  Balanced Accuracy  0.774713


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.